In [1]:
import pandas as pd
import numpy as np

### Загрузим CSV с GitHub

In [537]:
df_base = pd.read_csv(r"https://raw.githubusercontent.com/YBI-Foundation/Dataset/main/Pokemon.csv")
df_base = df_base.drop('SN', axis = 1)
df_base

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,DiancieMega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,HoopaHoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,HoopaHoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


In [539]:
df_base.isna().sum()

Name             0
Type 1           0
Type 2         386
HP               0
Attack           0
Defense          0
Sp. Attack       0
Sp. Defense      0
Speed            0
Generation       0
Legendary        0
dtype: int64

### 1. Определить, в длинной или широкой форме представлен CSV


Форма - широкая
### 2. Преобразовать широкую в длинную форму

In [541]:
df_long = df_base.copy()

In [543]:
df_long = (
    df_base
    .reset_index()
    .melt(
        id_vars = ['Name', 'Type 1', 'Type 2'],
        value_vars = ['HP', 'Attack', 'Defense', 'Sp. Attack', 'Sp. Defense', 'Speed'],
        var_name = 'Stats', 
        value_name = 'Amount'
    )
)
df_long

,Name,Type 1,Type 2,Stats,Amount
0,Bulbasaur,Grass,Poison,HP,45
1,Ivysaur,Grass,Poison,HP,60
2,Venusaur,Grass,Poison,HP,80
3,VenusaurMega Venusaur,Grass,Poison,HP,80
4,Charmander,Fire,NaN,HP,39
...,...,...,...,...,...
4795,Diancie,Rock,Fairy,Speed,50
4796,DiancieMega Diancie,Rock,Fairy,Speed,110
4797,HoopaHoopa Confined,Psychic,Ghost,Speed,70
4798,HoopaHoopa Unbound,Psychic,Dark,Speed,80


In [545]:
df_long.pivot(index = ['Name'], columns = 'Stats', values = 'Amount')

Stats,Attack,Defense,HP,Sp. Attack,Sp. Defense,Speed
Name,,,,,,
Abomasnow,92,75,90,92,85,60
AbomasnowMega Abomasnow,132,105,90,132,105,30
Abra,20,15,25,105,55,90
Absol,130,60,65,75,60,75
AbsolMega Absol,150,60,65,115,60,115
...,...,...,...,...,...,...
Zoroark,105,60,60,120,60,105
Zorua,65,40,40,80,40,65
Zubat,45,35,40,30,40,55


In [553]:
# Определим самого сильного покемона, добавив результирующую колонку 'Total_strength', где перемножаются все статы, 
#а уровень поколения прибавляется

df_long_total = df_long.copy()

df_long_total['Total_strength'] = (    
        (df_base['Attack'] * df_base['Defense'] * df_base['HP'] * df_base['Sp. Attack'] 
        * df_base['Sp. Defense'] * df_base['Speed'] + df_base['Generation'])/ 10**10
)

# Отсортируем покемонов по получившемуся показателю общей силы. 
# По данной методике ТОП-3 самых сильных покемонов - MewtwoMega Mewtwo X, RayquazaMega Rayquaza, MewtwoMega Mewtwo Y

df_long_total.sort_values('Total_strength', ascending = False).head(10).reset_index(drop = True)

,Name,Type 1,Type 2,Stats,Amount,Total_strength
0,MewtwoMega Mewtwo X,Psychic,Fighting,HP,106,403.202800
1,RayquazaMega Rayquaza,Dragon,Flying,HP,105,391.230000
2,MewtwoMega Mewtwo Y,Psychic,NaN,HP,106,362.748960
3,KyogrePrimal Kyogre,Water,NaN,HP,100,349.920000
4,GroudonPrimal Groudon,Ground,Fire,HP,100,349.920000
5,Arceus,Normal,NaN,HP,120,298.598400
6,SalamenceMega Salamence,Dragon,Flying,HP,95,232.081200
7,LatiasMega Latias,Dragon,Psychic,HP,80,221.760000
8,GarchompMega Garchomp,Dragon,Ground,HP,108,221.443632
9,MetagrossMega Metagross,Steel,Psychic,HP,80,221.067000


### 3. Выбрать одну числовую переменную и использовать функцию pd.pivot_table()


In [555]:
# Определим самый сильный Тип 2 покемона по средним показателям статов. ТОП-3: Dragon, Fighting, Ice
(
    df_long
    .pivot_table(values = 'Amount', 
                 index = ['Type 2'], 
                 columns = 'Stats', 
                 aggfunc = 'mean',
                 margins = True, 
                 margins_name = 'Total')
    .sort_values('Total', ascending = False)
    .drop('Total', axis = 0)    
    .head (50)
)

Stats,Attack,Defense,HP,Sp. Attack,Sp. Defense,Speed,Total
Type 2,,,,,,,
Dragon,94.444444,87.055556,82.166667,98.500000,83.444444,80.555556,87.694444
Fighting,112.846154,82.500000,79.461538,81.269231,82.384615,87.384615,87.641026
Ice,98.000000,85.642857,90.000000,92.642857,84.785714,74.642857,87.619048
Fire,81.250000,80.166667,71.250000,110.416667,83.500000,79.666667,84.375000
Steel,92.590909,104.636364,64.636364,77.727273,86.272727,59.363636,80.871212
Dark,109.800000,72.050000,75.550000,81.800000,69.850000,75.350000,80.733333
Psychic,74.696970,87.030303,72.212121,88.030303,87.606061,69.484848,79.843434
Electric,72.666667,68.333333,88.166667,81.166667,73.500000,71.500000,75.888889
Flying,80.288660,68.309278,71.391753,75.628866,71.206186,85.721649,75.424399


### Выбрать две другие числовые переменные и использовать цепочку методов .groupby().agg()


In [445]:
# Определим самый сильный Тип 1 покемона по Атаке (в среднем). ТОП-3: Dragon, Fighting, Ground
(
    df_base
    .groupby('Type 1')['Attack']
    .agg('mean')
    .sort_values(ascending = False)
    .head(10)
)

Type 1
Dragon      112.125000
Fighting     96.777778
Ground       95.750000
Rock         92.863636
Steel        92.703704
Dark         88.387097
Fire         84.769231
Flying       78.750000
Poison       74.678571
Water        74.151786
Name: Attack, dtype: float64

In [483]:
# Определим у каких покемонов Тип 2 максимальные показатели по Здоровью. ТОП-3: Dragon, Flying, Fairy
(
    df_base
    .groupby('Type 2')
    .agg({'HP': 'max'})
    .sort_values('HP', ascending = False)
    .head()
)

,HP
Type 2,
Dragon,150
Flying,150
Fairy,140
Ice,130
Electric,125


In [430]:
# Определим статистические характеристики показателя Защита по Тип 1, Тип 2 покемонтов
(
    df_base
    .groupby(['Type 1', 'Type 2'])
    .agg({'Defense': [('Сумма', 'sum'), ('Cреднее', 'mean'), ('Минимум', 'min'), ('Максимум', 'max'), ('Счет', 'count')]})
)

Defense                                  
                  Сумма     Cреднее Минимум Максимум Счет
Type 1 Type 2                                            
Bug    Electric     110   55.000000      50       60    2
       Fighting     190   95.000000      75      115    2
       Fire         120   60.000000      55       65    2
       Flying       862   61.571429      30      120   14
       Ghost         45   45.000000      45       45    1
...                 ...         ...     ...      ...  ...
Water  Ice          340  113.333333      80      180    3
       Poison       175   58.333333      35       75    3
       Psychic      520  104.000000      65      180    5
       Rock         451  112.750000      85      133    4
       Steel         88   88.000000      88       88    1

[136 rows x 5 columns]

In [599]:
# Сделаем аггригирование по двум переменным Amount и Total_strength для типов 1 и 2.
# Данная выгрузка позволит более полноценно определить уровень силы покемонов. 
# Можно применять разные методики оценки силы покемонов, не только по первичным статам, но и с учетом их совокупного показателя Total_strength

df_long_fin = (df_long_total
    .groupby(['Type 1', 'Type 2'])
    .agg(
        {
            'Amount': [('Минимум', 'min'), ('Cреднее', 'mean'), ('Максимум','max')],
            'Total_strength' : [('Минимум', 'min'), ('Cреднее', 'mean'), ('Максимум','max')]
        }
    )
)
df_long_fin

Amount                     Total_strength             \
                Минимум    Cреднее Максимум        Минимум    Cреднее   
Type 1 Type 2                                                           
Bug    Electric      47  65.916667      108       2.176688  11.252159   
       Fighting      40  91.666667      185      24.225000  38.919000   
       Fire          50  75.833333      135       4.242563  25.616344   
       Flying        20  69.916667      160       0.333396  15.725123   
       Ghost          1  39.333333       90       0.014580   0.014580   
...                 ...        ...      ...            ...        ...   
Water  Ice           45  85.277778      180      22.892625  29.727875   
       Poison        35  71.111111      120       1.960000  16.270701   
       Psychic       15  80.166667      180       0.912600  23.187195   
       Rock          22  71.458333      133       2.276337  11.105349   
       Steel         60  88.333333      111      42.761803  42.761803   

                            
                  Максимум  
Type 1 Type 2               
Bug    Electric  20.327630  
       Fighting  53.613000  
       Fire      46.990125  
       Flying    74.262825  
       Ghost      0.014580  
...                    ...  
Water  Ice       42.829800  
       Poison    34.944000  
       Psychic   40.014000  
       Rock      18.822375  
       Steel     42.761803  

[136 rows x 6 columns]

In [591]:
import openpyxl
df_long_fin.to_excel('Pokemon_fin.xlsx', engine='openpyxl')